In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Practice').getOrCreate()

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 5, Finished, Available, Finished)

## **Silver Notebook**

In [4]:
import notebookutils

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 6, Finished, Available, Finished)

In [5]:
notebookutils.fs.ls("Files")

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 7, Finished, Available, Finished)

[FileInfo(path=abfss://eaa0679f-3c23-4f37-a86d-6edc367f16b6@onelake.dfs.fabric.microsoft.com/e822713f-d0a4-496d-be64-9682427ffa70/Files/storeDataShortcut, name=storeDataShortcut, size=0)]

### **Data Ingestion - Reading using PySpark**

**Same Lakehouse**

In [6]:
df = spark.read.format('csv')\
               .option('header', True)\
               .option('inferSchema', True)\
               .load('Files/storeDataShortcut')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 8, Finished, Available, Finished)

In [7]:
display(df)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 97b4f16e-1394-4cb4-bacc-a31f8110de72)

**Reading file from different LH(Bronze)**

In [8]:
df_customer = spark.read.format('delta')\
                        .load('abfss://LearnFabric@onelake.dfs.fabric.microsoft.com/BronzeLH.Lakehouse/Tables/customer_dim')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 10, Finished, Available, Finished)

In [9]:
display(df_customer)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 87d0381f-4a6f-4924-81c4-8ecc4d06eed2)

**Another way to read a table**

In [10]:
df_cust = spark.read.table("BronzeLH.customer_dim")

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 12, Finished, Available, Finished)

In [11]:
display(df_cust)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 88e3a353-cad0-4a23-bf15-f56e7981ff17)

## **Transformation**

In [12]:
from pyspark.sql.functions import *


StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 14, Finished, Available, Finished)

In [13]:
df_cust = df_cust.withColumn('firstname', split(col('name'), ' '))

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 15, Finished, Available, Finished)

In [14]:
df_cust = df_cust.withColumn('firstname', split(col('name'), ' ')[0])\
                 .withColumn('lastname', split(col('name'), ' ')[1])

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 16, Finished, Available, Finished)

In [15]:
display(df_cust)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 73378cb6-8ceb-4c18-886b-59de3c055329)

### Replacing NULL with na

In [16]:
df_cust= df_cust.fillna({'lastname': 'na'})

display(df_cust)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 40f8c4eb-eeb3-4f31-befd-78878acba3bb)

**Data Writing**

External Table

In [17]:
df_cust.write.format('delta')\
             .mode('overwrite')\
             .option('path', 'abfss://LearnFabric@onelake.dfs.fabric.microsoft.com/SilverLH.Lakehouse/Files/ExternalTable')\
             .saveAsTable('SilverLH.ExtTbl')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 19, Finished, Available, Finished)

Managed table

In [18]:
df_cust.write.format('delta')\
             .mode('overwrite')\
             .saveAsTable('SilverLH.customers_dim')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 20, Finished, Available, Finished)

**Fact Table**

In [19]:
df_fact = spark.read.table('BronzeLH.fact_table')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 21, Finished, Available, Finished)

In [21]:
display(df_fact)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 187064e0-6f54-44fc-b8a5-26582b76adb1)

In [24]:
from pyspark.sql.types import *

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 26, Finished, Available, Finished)

In [25]:
df_fact = df_fact.withColumn('quantity', col('quantity').cast(IntegerType()))\
                 .withColumn('unit_price', col('unit_price').cast(IntegerType()))\
                 .withColumn('total_price', col('total_price').cast(IntegerType()))

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 27, Finished, Available, Finished)

In [27]:
df_fact.printSchema()

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 29, Finished, Available, Finished)

root
 |-- payment_key: string (nullable = true)
 |-- coustomer_key: string (nullable = true)
 |-- time_key: string (nullable = true)
 |-- item_key: string (nullable = true)
 |-- store_key: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit: string (nullable = true)
 |-- unit_price: integer (nullable = true)
 |-- total_price: integer (nullable = true)



In [28]:
display(df_fact)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 30, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fc9ab9b4-0547-4ee1-947d-ba80afb84cfa)

In [29]:
df_fact.write.format('delta')\
             .mode('overwrite')\
             .saveAsTable('SilverLH.fact_table')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 31, Finished, Available, Finished)

**Store Dim**

In [30]:
df_store = spark.read.table('BronzeLH.store_dim')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 32, Finished, Available, Finished)

In [31]:
display(df_store)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 33, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 49e449a7-efb7-4fb9-9f2b-73379d4e2274)

In [35]:
df_store = df_store.withColumn('address', concat(col('district'), lit('-'), col('upazila')))

df_store = df_store.drop('district', 'upazila')

display(df_store)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 15732258-7578-4bff-af75-e04140f18567)

In [41]:
df_store.write.format('delta')\
              .mode('overwrite')\
              .saveAsTable('store_dim') 

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 43, Finished, Available, Finished)

**Trans_dim**

In [36]:
df_trans = spark.read.table('BronzeLH.Trans_dim')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 38, Finished, Available, Finished)

In [37]:
display(df_trans)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 39, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a65d771b-0f94-452a-96d2-5528f1927e8a)

In [39]:
df_trans = df_trans.withColumn('bank_name', regexp_replace(col('bank_name'), 'None', 'Not Available'))

display(df_trans)

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 41, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a99e5429-897e-4f71-a3c3-7768b7957cfc)

In [40]:
df_trans.write.format('delta')\
              .mode('overwrite')\
              .saveAsTable('SilverLH.trans_dim')

StatementMeta(, 79db0f93-12be-4ea5-81c3-267ba229311e, 42, Finished, Available, Finished)